# Volition Font Processor (Read/Write) 
###### for Saints Row (2006)

## VF3 Specification
- Header: 0xBC
    - 0x00 - ID (4) - (*LE*)**VFNT/TNFV**(*BE*)
    - 0x04 - Version (4)
    - 0x08 - NumberOfCharacters (4)
    - 0x0C - FirstAscii (4)
    - 0x10 - Width (4)
    - 0x14 - Height (2)
    - 0x16 - RenderHeight (2)
    - 0x18 - BaselineOffset (4)
    - 0x1C - CharacterSpacing (4)
    - 0x20 - NumberOfKerningPairs (4)
    - 0x24 - VerticalOffset (2)
    - 0x26 - HorizontalOffset (2)
    - 0x28 - PegName (0x40)
    - 0x68 - BitmapName (0x54?)
- KerningPairs: 0x04 * NumberOfKerningPairs
    - 0x00 - Char1 (0x20 ~ 0xFF)
    - 0x01 - Char2 (0x20 ~ 0xFF)
    - 0x02 - Offset
    - 0x03 - Padding
- Characters: 0x10 * NumberOfCharacters
    - 0x00 - Spacing (4)
    - 0x04 - ByteWidth (4)
    - 0x08 - Offset (4)
    - 0x0C - KerningEntry (2)
    - Ox0E - UserData (2)
- U Coords: 0x04 * NumberOfCharacters
- V Coords: 0x04 * NumberOfCharacters

Little-endian (LE) fonts are marked as .vf3_pc and have signature VFNT.
Big-endian (BE) fonts are marked as .vf3_xbox2 and have signature TNFV.

In [1]:
import os
import struct
import pandas as pd
from PIL import Image, ImageFont, ImageDraw
from fontTools.ttLib import TTFont

In [2]:
BG_NULL = os.path.join("fontforge","background.png")
BG_HALF = os.path.join("fontforge","background_half.png")
BG_BLACK =  os.path.join("fontforge","black.png")

NAMES = {
    'digital': ['px_digital'],
    'thin': ['px_thin','px_thinoutline'],
    'giant': ['px_giantvar','px_giantvar2','px_giantout']
}

FONTS = {
    'digital': 'Glasstown256.otf',
    'thin': 'Helvetica256.otf',
    'giant': 'Molot256.otf'
}

FONTSIZE = {
    'digital': 24,
    'thin': 24,
    'giant': 35
}

SETTINGS = {
    'digital':{
        'width':11, 
        'height':28, 
        'renderHeight':24, 
        'spacing':0, 
        'vOffset':0, 
        'hOffset':0
    },
    'thin':{
        'width':22, 
        'height':29, 
        'renderHeight':36, 
        'spacing':-10, 
        'vOffset':-2, 
        'hOffset':-2
    },
    'giant':{
        'width':13, 
        'height':35, 
        'renderHeight':48, 
        'spacing':-10, 
        'vOffset':-2, 
        'hOffset':-2
    }
}

In [3]:
def unpackInt(data, size='c', order='little'):
    res = (
        struct
        .unpack('>'+size, data)[0]
    )
    return res

def unpackTextSmall(data, size, sizenum=1, order='little'):
    res = (
        struct
        .unpack('>'+size, data)[0]
        .to_bytes(sizenum,order)
        .rstrip(b'\x00')
        .decode()
    )
    return res

def unpackTextBig(data):
    res = (
        data
        .rstrip(b'\x00')
        .decode()
    )
    return res

In [4]:
def generateHelper(vf3_file, dds_file=None, png_file=None):
    vData = {}
    
    with open(vf3_file,"rb") as vf3:
        vData['ID'] = unpackTextSmall(vf3.read(4), 'i', 4)
        vData['Version'] = unpackInt(vf3.read(4),'i')
        vData['NumberOfCharacters'] = unpackInt(vf3.read(4),'i')
        vData['FirstAscii'] = unpackTextSmall(vf3.read(4),'i', 4)
        vData['Width'] = unpackInt(vf3.read(4),'i')
        vData['Height'] = unpackInt(vf3.read(2),'h')
        vData['RenderHeight'] = unpackInt(vf3.read(2),'h')
        vData['BaselineOffset'] = unpackInt(vf3.read(4),'i')
        vData['CharacterSpacing'] = unpackInt(vf3.read(4),'i')
        vData['NumberOfKerningPairs'] = unpackInt(vf3.read(4),'i')
        vData['VerticalOffset'] = unpackInt(vf3.read(2),'h')
        vData['HorizontalOffset'] = unpackInt(vf3.read(2),'h')
        vData['PegName'] = unpackTextBig(vf3.read(0x40))
        vData['BitmapName'] = unpackTextBig(vf3.read(0x54))
        vData['KerningData'] = [
            {
                'Char1':   unpackInt(vf3.read(1),'c'),
                'Char2':   unpackInt(vf3.read(1),'c'),
                'Offset':  unpackInt(vf3.read(1),'b'),
                'Padding': unpackInt(vf3.read(1),'b')
            }
            for i in range(vData['NumberOfKerningPairs'])]
        vData['CharacterData'] = [
            {
                'Char': i+0x20,
                'Symbol': (i+0x20).to_bytes(1, byteorder='little'),
                'Spacing': unpackInt(vf3.read(4),'i'),
                'ByteWidth': unpackInt(vf3.read(4),'i'),
                'Offset': unpackInt(vf3.read(4),'i'),
                'KerningEntry': unpackInt(vf3.read(2),'h'),
                'UserData': unpackInt(vf3.read(2),'h')
            }
            for i in range(vData['NumberOfCharacters'])
        ]
        vData['BM_U'] = [
            unpackInt(vf3.read(4),'i') 
            for i in range(vData['NumberOfCharacters'])
        ]
        vData['BM_V'] = [
            unpackInt(vf3.read(4),'i') 
            for i in range(vData['NumberOfCharacters'])
        ]
        
        print('File:', vf3_file)
        print('ID:', vData['ID'])
        print('NumberOfCharacters:', vData['NumberOfCharacters'])
        print('Width:', vData['Width'])
        print('Height:', vData['Height'])
        print('RenderHeight:', vData['RenderHeight'])
        print('BaselineOffset:', vData['BaselineOffset'])
        print('CharacterSpacing:', vData['CharacterSpacing'])
        print('NumberOfKerningPairs:', vData['NumberOfKerningPairs'])
        print('VerticalOffset:', vData['VerticalOffset'])
        print('HorizontalOffset:', vData['HorizontalOffset'])
        print('PegName:', vData['PegName'])
        print('BitmapName:', vData['BitmapName'])
        print("===== ===== ===== ===== =====")
        
        if dds_file is None or png_file is None:
            return vData
        else:
            image = Image.open(dds_file)
            draw = ImageDraw.Draw(image)

            dfChar = pd.DataFrame(vData['CharacterData'])

            dfChar = pd.concat([
                dfChar,
                pd.Series(vData['BM_U'], name='U'),
                pd.Series(vData['BM_V'], name='V'),
                pd.Series(vData['Width'], name='W'),
                pd.Series(vData['Height'], name='H'),
                pd.Series(vData['RenderHeight'], name='RH')
            ], axis=1).fillna(method='ffill')

            for i in dfChar.iterrows():
                U = i[1]['U']
                V = i[1]['V']
                # W = i[1]['W']
                W = i[1]['ByteWidth']
                H = i[1]['RH']        
                draw.rectangle((U, V, U+W, V+H), outline="purple")
            image.save(png_file)
            
            return vData

In [5]:
def char_in_font(unicode_char, font):
    for cmap in font['cmap'].tables:
        if cmap.isUnicode():
            if ord(unicode_char) in cmap.cmap:
                return True
    return False

***

### Symbols in Thin font
*All symbols are having ByteWidth/Spacing of 40.*
- 127 0x7F - A Button


- 129 0x81 - B Button
- 130 0x82 - Y Button
- 131 0x83 - X Button
- 132 0x84 - LT Button


- 134 0x86 - RT Button
- 135 0x87 - LB Button
- 136 0x88 - RB Button
- 137 0x89 - LStick


- 139 0x8B - RStick


- 141 0x8D - Left
- 142 0x8E - Down-Left
- 143 0x8F - Down
- 144 0x90 - Down-Right


- 149 0x95 - Right
- 150 0x96 - Up-Right
- 151 0x97 - Up
- 152 0x98 - Up-Left


- 155 0x9B - D-Pad


- 157 0x9D - Infinity Sign

In [6]:
def setHeader(fontType):
    vSet = SETTINGS[fontType]
    vData = {}
    vData['ID'] = 'TNFV'
    vData['Version'] = 2
    vData['NumberOfCharacters'] = 224
    vData['FirstAscii'] = 0x20
    vData['Width'] = vSet['width']
    vData['Height'] = vSet['height']
    vData['RenderHeight'] = vSet['renderHeight']
    vData['BaselineOffset'] = 0
    vData['CharacterSpacing'] = vSet['spacing']
    vData['NumberOfKerningPairs'] = 4
    vData['VerticalOffset'] = vSet['vOffset']
    vData['HorizontalOffset'] = vSet['hOffset']
    vData['PegName'] = [NAMES[fontType][i]+'.peg' for i in range(len(NAMES[fontType]))]
    vData['BitmapName'] = [NAMES[fontType][i]+'_nobdr.tga' for i in range(len(NAMES[fontType]))]
    vData['KerningData'] = [
        {
            'Char1':   '\r',
            'Char2':   '0',
            'Offset':  0,
            'Padding': -51    
        },
        {
            'Char1':   '\r',
            'Char2':   '1',
            'Offset':  0,
            'Padding': -51    
        },
        {
            'Char1':   '\r',
            'Char2':   '2',
            'Offset':  0,
            'Padding': -51    
        },
        {
            'Char1':   '\r',
            'Char2':   '3',
            'Offset':  0,
            'Padding': -51    
        }
    ]
    vData['CharacterData'] = []
    vData['BM_U'] = []
    vData['BM_V'] = []
    return vData

In [21]:
def generateFont(fontType):
    vSet = SETTINGS[fontType]

    # For Giant - omitting lowercase letters
    # CharOmit1 = range(0x61, 0xC0)
    # CharOmit2 = range(0xE0, 0x100)
    # For Thin - replacing chars with icons
    CharSymbols = [127,129,130,131,132,
                   134,135,136,137,139,
                   141,142,143,144,149,
                   150,151,152,155,157]
    # For Giant - doubling non-english chars
    CharCaps = list(range(224,256))
    vData = setHeader(fontType)
    vFont = TTFont(os.path.join('fontforge',FONTS[fontType]))
    
    # For each fontfile in header...
    for k, _ in enumerate(NAMES[fontType]):
        # Cleaning buffers before each iteration
        PenX = 2
        PenY = 1
        
        saveX = 2
        saveY = 1
        
        Spacing = 10 if fontType != 'digital' else 2
        vData['CharacterData'] = []
        vData['BM_U'] = []
        vData['BM_V'] = []
        
        image = Image.open(BG_NULL) if fontType != 'digital' else Image.open(BG_HALF)
        draw = ImageDraw.Draw(image)
        if fontType == 'digital':
            draw.fontmode = '1'
        font = ImageFont.truetype(
            os.path.join('fontforge',FONTS[fontType]), 
            FONTSIZE[fontType])
        
        # For each symbol...
        for i in range(0x20,0x100):
            char = chr(i)
            W = font.getlength(char)
            H = vSet['renderHeight']
            if char_in_font(char, vFont):
                if i == 224 and fontType == 'giant':
                    print('Polo!')
                    PenX = saveX
                    PenY = saveY
                
                bbox = font.getbbox(char)
                if PenX+bbox[2] > 510:
                    PenX = 2
                    PenY += vSet['renderHeight'] + 3
                BW = W+Spacing
                
                if not (i in CharCaps and fontType == 'giant'):
                    print('Symbol', char, '- NORMAL', int(PenX), int(PenY))
                    draw.text((PenX+2, PenY), char, font=font)
                else:
                    print('Symbol', char, '- DOUBLE', int(PenX), int(PenY))

                # Character Write
                vData['CharacterData'].append(
                    {
                        'Spacing': BW,
                        'ByteWidth': BW,
                        'Offset': 0,
                        'KerningEntry': -1,
                        'UserData': 0
                    }        
                )
                vData['BM_U'].append(PenX)
                vData['BM_V'].append(PenY)
                
                if i == 192 and fontType == 'giant':
                    print('Marco!')
                    saveX = PenX
                    saveY = PenY

                # Pen Shift
                PenX += W+1+Spacing
                if PenX > 512:
                    PenX = 2
                    PenY += vSet['renderHeight'] + 3
            elif i in CharSymbols and fontType == 'thin':
                # Painting icons into the bitmap
                bbox = (0,0,40,36)
                if PenX+bbox[2] > 510:
                    PenX = 2
                    PenY += vSet['renderHeight'] + 3
                BW = 40
                print('Symbol', char, '- ICON', int(PenX), int(PenY))
                with Image.open(os.path.join('bmaps',str(i)+'.png')) as im:
                    image.paste(im, (int(PenX)+bbox[0], int(PenY)+bbox[1], 
                                     int(PenX)+bbox[2], int(PenY)+bbox[3]))
                                
                # Character Write
                vData['CharacterData'].append(
                    {
                        'Spacing': BW,
                        'ByteWidth': BW,
                        'Offset': 0,
                        'KerningEntry': -1,
                        'UserData': 0
                    }        
                )
                vData['BM_U'].append(PenX)
                vData['BM_V'].append(PenY)
                
                # Pen Shift
                PenX += 41
                if PenX > 512:
                    PenX = 2
                    PenY += vSet['renderHeight'] + 3
        
            else:
                print('Symbol', char, '- MISSING', int(PenX), int(PenY))
                BW = 1

                # Character Write
                vData['CharacterData'].append(
                    {
                        'Spacing': BW,
                        'ByteWidth': BW,
                        'Offset': 0,
                        'KerningEntry': -1,
                        'UserData': 0
                    }        
                )
                vData['BM_U'].append(PenX)
                vData['BM_V'].append(PenY)

                # Pen Shift
                PenX += 2
                if PenX > 510:
                    PenX = 2
                    PenY += vSet['renderHeight'] + 3

        image.save(os.path.join('fontforge',NAMES[fontType][k]+'_nobdr.png'))
                                
        # Запись в vf3.xbox2
        with open(os.path.join('vf3',NAMES[fontType][k]+'.vf3_xbox2'),'wb') as vf3:
            vf3.write(vData['ID'].encode('latin-1'))
            vf3.write(struct.pack('>i',vData['Version']))
            vf3.write(struct.pack('>i',vData['NumberOfCharacters']))
            vf3.write(struct.pack('>i',vData['FirstAscii']))
            vf3.write(struct.pack('>i',vData['Width']))
            vf3.write(struct.pack('>h',vData['Height']))
            vf3.write(struct.pack('>h',vData['RenderHeight']))
            vf3.write(struct.pack('>i',vData['BaselineOffset']))
            vf3.write(struct.pack('>i',vData['CharacterSpacing']))
            vf3.write(struct.pack('>i',vData['NumberOfKerningPairs']))
            vf3.write(struct.pack('>h',vData['VerticalOffset']))
            vf3.write(struct.pack('>h',vData['HorizontalOffset']))
            vf3.write(struct.pack('>64s',vData['PegName'][k].encode('latin-1')))
            vf3.write(struct.pack('>84s',vData['BitmapName'][k].encode('latin-1')))
            for i in vData['KerningData']:
                vf3.write(struct.pack('>ccbb',
                                      i['Char1'].encode('latin-1'),i['Char2'].encode('latin-1'),
                                      i['Offset'],i['Padding']))
            for i in vData['CharacterData']:
                vf3.write(struct.pack('>iiihh',
                                      int(i['Spacing']),int(i['ByteWidth']),    
                                      i['Offset'],i['KerningEntry'],i['UserData']))
            for i in vData['BM_U']:
                vf3.write(struct.pack('>i',int(i)))
            for i in vData['BM_V']:
                vf3.write(struct.pack('>i',int(i)))
            print ('Processing {0} complete.'.format(os.path.join('vf3',NAMES[fontType][k]+'.vf3_xbox2')))
    print ('All done!')
    return 0

***

In [22]:
generateFont('giant')

Symbol   - NORMAL 2 1
Symbol ! - NORMAL 30 1
Symbol " - NORMAL 52 1
Symbol # - NORMAL 82 1
Symbol $ - NORMAL 119 1
Symbol % - NORMAL 148 1
Symbol & - NORMAL 183 1
Symbol ' - NORMAL 215 1
Symbol ( - NORMAL 236 1
Symbol ) - NORMAL 256 1
Symbol * - NORMAL 276 1
Symbol + - NORMAL 303 1
Symbol , - NORMAL 333 1
Symbol - - NORMAL 354 1
Symbol . - NORMAL 384 1
Symbol / - NORMAL 405 1
Symbol 0 - NORMAL 434 1
Symbol 1 - NORMAL 465 1
Symbol 2 - NORMAL 486 1
Symbol 3 - NORMAL 2 52
Symbol 4 - NORMAL 32 52
Symbol 5 - NORMAL 64 52
Symbol 6 - NORMAL 95 52
Symbol 7 - NORMAL 126 52
Symbol 8 - NORMAL 153 52
Symbol 9 - NORMAL 184 52
Symbol : - NORMAL 215 52
Symbol ; - NORMAL 236 52
Symbol < - NORMAL 257 52
Symbol = - NORMAL 282 52
Symbol > - NORMAL 313 52
Symbol ? - NORMAL 338 52
Symbol @ - NORMAL 367 52
Symbol A - NORMAL 403 52
Symbol B - NORMAL 435 52
Symbol C - NORMAL 467 52
Symbol D - NORMAL 2 103
Symbol E - NORMAL 34 103
Symbol F - NORMAL 63 103
Symbol G - NORMAL 92 103
Symbol H - NORMAL 124 103
Symb

0